In [1]:
from carsus import init_db
session = init_db("sqlite://")
session.commit()

Initializing the database
Ingesting basic atomic data


In [2]:
from carsus.io.nist import NISTWeightsCompIngester
ingester = NISTWeightsCompIngester()
ingester.download()
ingester.ingest(session)

Ingesting atomic weights


In [5]:
# Lets put out a table with z, weight in solar masses, z**2 + 5

# First we need to write a query
# Unit conversion is done on the DB side!
from carsus.model import Atom, AtomicWeight
from astropy import units as u
q = session.query(Atom.atomic_number.label("z"),
                  AtomicWeight.quantity.to(u.solMass).value.label("weight_solMass")).\
                  join(Atom.quantities.of_type(AtomicWeight))

# Then we use pandas to read the query into a DataFrame
from pandas import read_sql_query
df = read_sql_query(q.selectable, session.bind)

# Once we have the data we can compute things
df["z**2+5"] = df["z"]**2 + 5

print df.head(5)

   z  weight_solMass  z**2+5
0  1    8.414768e-58       6
1  2    3.341449e-57       9
2  3    5.816602e-57      14
3  4    7.523543e-57      21
4  5    9.027316e-57      30


In [6]:
# Another example
q = q.session.query(Atom.atomic_number.label("z"),
                AtomicWeight.quantity.value.label("weight_u")).\
                join(Atom.quantities.of_type(AtomicWeight)).\
                filter(AtomicWeight.quantity < 15*u.u)
df2 = read_sql_query(q.selectable, session.bind)
print df2

   z   weight_u
0  1   1.007975
1  2   4.002602
2  3   6.967500
3  4   9.012183
4  5  10.813500
5  6  12.010600
6  7  14.006855
